In [ ]:
# default_exp datasets.movielens

# MovieLens Dataset
> Implementation of MovieLens datasets.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import pickle
import tempfile
from pathlib import Path
from collections import defaultdict
import shutil
from datetime import date
import os.path as osp
import os

import torch.utils.data

from recohut.datasets.base import Dataset, RatingDataset
from recohut.utils.common_utils import download_url, extract_zip

## ML1m Rating Dataset

In [ ]:
#export
class ML1mRatingDataset(RatingDataset, torch.utils.data.Dataset):
    url = "http://files.grouplens.org/datasets/movielens/ml-1m.zip"

    @property
    def raw_file_names(self):
        return 'ratings.dat'

    def download(self):
        path = download_url(self.url, self.raw_dir)
        extract_zip(path, self.raw_dir)
        from shutil import move, rmtree
        move(osp.join(self.raw_dir, 'ml-1m', self.raw_file_names), self.raw_dir)
        rmtree(osp.join(self.raw_dir, 'ml-1m'))
        os.unlink(path)

    def process(self):
        data = pd.read_csv(self.raw_paths[0], sep='::', header=None, engine='python').to_numpy()[:, :3]
        self.items = data[:, :2].astype(np.int) - 1  # -1 because ID begins from 1
        self.targets = self.__preprocess_target(data[:, 2]).astype(np.float32)
        self.field_dims = np.max(self.items, axis=0) + 1
        self.user_field_idx = np.array((0, ), dtype=np.long)
        self.item_field_idx = np.array((1,), dtype=np.long)

    def __len__(self):
        return self.targets.shape[0]

    def __getitem__(self, index):
        return self.items[index], self.targets[index]

    def __preprocess_target(self, target):
        # treat samples with a rating less than 3 as negative samples
        target[target <= 3] = 0
        target[target > 3] = 1
        return target

In [ ]:
ds = ML1mRatingDataset(root='/content/ML1m', min_uc=10, min_sc=5)

Extracting /content/ML1m/raw/ml-1m.zip
Processing...
Done!


## ML100k Dataset

In [ ]:
#export
class ML100kDataset(Dataset):
    url = 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'
    
    def __init__(self, root):
        super().__init__(root)
    
    @property
    def raw_file_names(self) -> str:
        return ['u1.base', 'u1.test', 'u4.test', 'allbut.pl', 'u.item', 
                'ua.test', 'u.occupation', 'u3.test', 'u5.base', 'ub.test', 
                'u2.test', 'u3.base', 'u.genre', 'u.data', 'u4.base', 
                'u5.test', 'u.info', 'README', 'ub.base', 'mku.sh', 'u2.base', 
                'u.user', 'ua.base']

    @property
    def processed_file_names(self) -> str:
        raise NotImplementedError

    def download(self):
        path = download_url(self.url, self.raw_dir)
        extract_zip(path, self.raw_dir)
        from shutil import move, rmtree
        file_names = os.listdir(osp.join(self.raw_dir, 'ml-100k'))   
        for file_name in file_names:
            move(osp.join(self.raw_dir, 'ml-100k', file_name), self.raw_dir)
        rmtree(osp.join(self.raw_dir, 'ml-100k'))
        os.unlink(path)

    def process(self):
        raise NotImplementedError

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d

Author: Sparsh A.

Last updated: 2021-12-21 09:41:25

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.104+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

google : 2.0.3
numpy  : 1.19.5
pandas : 1.1.5
IPython: 5.5.0

